In [1]:
using PorousMaterials
using PyPlot
using DataFrames
using CSV

define variables

In [ ]:
σ_y = 
n_Xe = 

P_range = range(0.0, 200.0, length=200)

read in density of xenon data (in bulk and adsorbed conditions)

In [16]:
const df_xtal = CSV.read("df_crystal.csv")

ErrorException: cannot declare df_xtal constant; it already has a value

In [15]:
const df_xe = vcat(CSV.read(joinpath("data", "NIST_data", "low_pressure_xenon_NIST_data.txt"))[2:end, 1:3],
             CSV.read(joinpath("data", "NIST_data", "xenon_NIST_data.txt"))[:, 1:3])
sort!(df_xe, Symbol("Pressure (bar)"))

,Temperature (K),Pressure (bar),Density (mol/l)
,Float64,Float64,Float64
1,298.0,0.0,0.0
2,298.0,0.0025,0.0001009
3,298.0,0.005,0.0002018
4,298.0,0.0075,0.00030271
5,298.0,0.01,0.00040362
6,298.0,0.0125,0.00050453
7,298.0,0.015,0.00060544
8,298.0,0.0175,0.00070636
9,298.0,0.02,0.00080728


melanie: see `inter_linear` http://juliamath.github.io/Interpolations.jl/latest/convenience-construction/

In [9]:
# return density of Xenon in MOF under Langmuir model
function ρ_Xe_ads(P::Float64, xtal::String)
    # get M, K from the crystal dataframe
    return M * K * P / (1 + K * P) # mol/m³
end

# return density of bulk Xenon gas
# this will read in and interpolate NIST data
function ρ_Xe(P::Float64)
    # interpolate data in df_xe
end

ρ_Xe (generic function with 1 method)

In [10]:
# radius of vessel needed
function r(P::Float64, xtal::String) # ads Xe storage
end

function r(P::Float64) # bulk Xe storage
end

# mass of tank material needed
function mₜ(P::Float64, xtal::String) # ads Xe storage
end

function mₜ(P::Float64)  # bulk Xe storage
end

# mass of adsorbent needed
function mₐ(P::Float64, xtal::String) # ads Xe storage (of course)
end

function P_opt(xtal::String) # ads Xe storage
    # use Optim.jl to find the minimum
end

function P_opt() # bulk Xe storage
end

mₐ (generic function with 1 method)